## Install lib

In [1]:
%%bash
rm -rf /usr/local/cuda
ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!stat /usr/local/cuda

  File: /usr/local/cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 34h/52d	Inode: 3028158     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2021-11-22 11:48:01.494879953 +0000
Modify: 2021-11-22 11:48:01.447879968 +0000
Change: 2021-11-22 11:48:01.447879968 +0000
 Birth: -


In [4]:
!pip uninstall triton

In [ ]:
!pip uninstall torch

In [5]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [7]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
clang-9 is already the newest version (1:9-2~ubuntu18.04.2).
llvm-9 is already the newest version (1:9-2~ubuntu18.04.2).
llvm-9-dev is already the newest version (1:9-2~ubuntu18.04.2).
llvm-9-tools is already the newest version (1:9-2~ubuntu18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [9]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Writing setup.sh


In [ ]:
!sh setup.sh

In [ ]:
!pip install triton==0.2.3

In [9]:
# !pip uninstall -y typing

In [ ]:
!pip install cpufeature

In [ ]:
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7

In [14]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
sparse_attn ............ [YES] ...... [OKAY]
transformer ............ [NO] ....... [OKAY]
stochastic_transformer . [NO] ....... [OKAY]
utils .................. [NO] ....... [OKAY]
--------------------------------------------------
DeepSpeed general environment info:
tor

In [15]:
import deepspeed.ops.sparse_attention.sparse_attn_op

In [1]:
!rm -rf ru-gpts

In [2]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 673, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 673 (delta 103), reused 135 (delta 80), pack-reused 505
Receiving objects: 100% (673/673), 402.50 KiB | 3.73 MiB/s, done.
Resolving deltas: 100% (403/403), done.


In [ ]:
!pip install transformers==3.5.1

In [18]:
!cp ru-gpts/src_utils/trainer_pt_utils.py /usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py

# Test model

### Load model

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sys
sys.path.append("ru-gpts/")

In [5]:
import os
os.environ["USE_DEEPSPEED"] = "1"

In [6]:
from src.xl_wrapper import RuGPT3XL

Note! seq_len is max sequence length for generation used in generation process. Max avialable seq_len is 2048 (in tokens).
Also inference takes around 10 Gb GPU memory.

In [7]:
# gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)
gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)
# model parallel group is not initialized - если не подключена gpu 

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
Use alternating sparse & dense attention layers


In [8]:
# gpt.model = gpt.model.float()

### Get logits

In [9]:
logits = gpt("Кто был президентом США в 2020? ").logits

In [10]:
type(logits), logits.shape

(torch.Tensor, torch.Size([1, 8, 50264]))

### Get loss

In [11]:
input_ids = [gpt.tokenizer("Кто был президентом США в 2020? ")['input_ids']]
labels = input_ids

In [12]:
import torch


with torch.no_grad():
    loss = gpt(input_ids=input_ids, labels=labels).loss

In [13]:
loss

[tensor(4.3879, device='cuda:0')]

### Simple generation

In [14]:
def filter_resuls(nr):
    return [x[:x.find("<|endoftext|>")] for x in nr]

Greedy decoding

In [15]:
filter_resuls(gpt.generate(
    "Кто был президентом США в 2020? ",
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Кто был президентом США в 2020? \nВ этом году выборы президента Соединенных Штатов Америки пройдут уже через несколько дней. И, как и всегда на протяжении последних лет (а это не первый раз), кандидаты будут бороться за право стать главой государства с помощь']

sample

In [16]:
filter_resuls(gpt.generate(
    "Кто был президентом США в 2020? ", do_sample=True, num_return_sequences=5,
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Кто был президентом США в 2020? \nНа этой неделе, после того как американские граждане избрали Трампа. Демократическая партия сделала заявление следующего содержания: "Мы не хотим этого человека на посту президента". Таким образом демократы дали понять что их кандидат',
 'Кто был президентом США в 2020? ______________________________________________________ 1) Гарри Трумэн 2)"Джордж Б. Вудро" 3)(1929-1994гг.) 4(1884 - 1981 гг.), 25 лет как ушел',
 'Кто был президентом США в 2020? \nЗа кого будет голосовать Америка после 11 января. В январе 2021 года истекает срок президентства Барака Обамы (у него осталось 40 дней). По традиции, кандидат от Республиканской партии должен выдвигаться последни',
 'Кто был президентом США в 2020? \n2022 год станет переломным для истории мира, и некоторые страны уже объявили о своих желаниях участвовать или проиграть предстоящие выборы. Но если посмотреть на другие кандидатуры кандидатов по сравнению с Трампом многи',
 'Кто был президентом США в 2020? \nК

### Top_k top_p filtering

In [17]:
filter_resuls(gpt.generate(
    "Александр Сергеевич Пушкин родился в ",
    top_k=5,
    top_p=0.95,
    temperature=1.2,
    num_return_sequences=5,
    do_sample=True,
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Александр Сергеевич Пушкин родился в \nСанкт-Петербурге (по некоторым данным - 23 апреля 1799 г.), но, как известно из школьного курса истории, умер на дуэли. \n В настоящее время принято считать датой его рождения 22 ноября по стодне',
 'Александр Сергеевич Пушкин родился в \nгороде Москве. В 1820 году, когда ему не было еще и шести лет отроду он поступил на службу к князю Николаю Борисовичу Юсупову (1801-1787). С этого времени жизн',
 'Александр Сергеевич Пушкин родился в \n1817 году, умер через несколько лет после смерти своей любимой матери - Екатерины. Отец его был человеком очень образованным и умным: он владел французским языком лучше всех своих сыновей; сам писал по-французски стихи ',
 'Александр Сергеевич Пушкин родился в \n1812 году, умер через два дня после рождения. Его детство и юность прошли среди друзей его отца – Александра Сергеевича Грибоедова; с детства они были неразлучны (Грибовские жили н',
 'Александр Сергеевич Пушкин родился в \nМоскве. В 1817 году окончил Ца

### Beamsearch

In [18]:
filter_resuls(gpt.generate(
    text="Александр Сергеевич Пушкин родился в ",
    max_length=50,
    num_beams=10,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
    num_return_sequences=5,
))

['Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать, урожденная Ганнибал, был',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать поэта, Мария Алексеевна Ганнибал',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать поэта, Наталья Николаевна \nПушкин',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать, урождённая Ганнибал, был',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \n